In [3]:
#导入数据
from numpy import float64
import pandas as pd
from dq_work import rw_file
import imp
imp.reload(rw_file)#再次引入模块，默认只有一次，调试程序时有用
read_file = rw_file.rw_csv()

#配置参数

company='上海'
date_Range='202207'
date_Range_all='20220701-0731'

#company='上海'
#date_Range='20220301-0331'

#需要准备的导入文件清单
######################################
in_file_dir='e:/python-dq/work_data/tmp_data/in/'
in_file='02_业务报表-销售报表-大类层级-供应商-'+company+ '-'+date_Range+'.csv'
in_file2='采购收入_'+company+ '_'+date_Range_all+'.csv'
in_file3='供应商券分摊_out'+company+ '-all-'+date_Range_all+'.csv'
in_file4='01-用劵分摊-商品层级-原版-'+company+ '-all-'+date_Range_all+'.csv'



######################################

#配置导出结果的目录与文件名
out_file_dir='e:/python-dq/work_data/tmp_data/out/'
out_file='10.供应商毛利分析_out_'+company+ '-'+date_Range_all+'.xlsx'




df_供应商原始数据 = read_file.r_csv_utf8(in_file_dir + in_file,0,0)
df_采购收入原始数据 = read_file.r_csv_ansi(in_file_dir + in_file2,0,0)
# 删选省区业绩的采购收入
bool2=~df_采购收入原始数据['扣项类型'].str.contains('装卸|搬运')
df_采购收入原始数据=df_采购收入原始数据[bool2]
df_券金额原始数据 = read_file.r_csv_utf8(in_file_dir + in_file3 ,0,0)
print('data in over')



data in over


In [4]:
print(df_供应商原始数据.columns)
print(df_采购收入原始数据.columns)
print(df_券金额原始数据.columns)


Index(['日期', '省级公司名称', '渠道', '平台', '模块编码', '模块名称', '品类编码', '品类名称', '大类编码',
       '大类名称', '供应商编码', '供应商业务码', '供应商名称', '月-销售数量', '月-销售净额(折前)', '月-销售净额',
       '月-销售成本', '月-折扣净额', '月-生鲜损耗', '月-补差', '月-销售毛利', '月-系统毛利额', '月-客流量',
       '年-销售数量', '年-销售净额(折前)', '年-销售净额', '年-销售成本', '年-折扣净额', '年-生鲜损耗', '年-补差',
       '年-销售毛利', '年-系统毛利额', '年-客流量'],
      dtype='object')
Index(['月份', '发生BU', '发生机构', '发生机构名称', '供应商编码', '供应商名称', '大类编码', '大类名称',
       '扣项ID', '扣项名称', '扣项类型', '金额', '税率'],
      dtype='object')
Index(['大类编码', '供应商业务码', '券分摊金额'], dtype='object')


In [9]:
df_券金额原始数据.rename(columns={'供应商业务码':'供应商编码'},inplace=True)

In [19]:
print(df_供应商原始数据['月-销售净额'].sum())
print(df_供应商原始数据['月-销售毛利'].sum())

print(df_采购收入原始数据['金额'].sum())
print(df_券金额原始数据[ '券分摊金额'].sum())

'''
121592244.41962
45462354.43
59896152.52862591
'''

112539903.087356
15891038.341931999
6154987.870000001
7352855.821251824


'\n121592244.41962\n45462354.43\n59896152.52862591\n'

In [11]:
g_销售_供应商 =df_供应商原始数据.groupby(['大类编码', '供应商编码'])
df_供应商_销售=pd.DataFrame(g_销售_供应商[['月-销售净额','月-销售毛利']].sum())
df_供应商_销售.reset_index(inplace=True)

g_采收_供应商 =df_采购收入原始数据.groupby(['大类编码', '供应商编码'])
df_供应商_采收=pd.DataFrame(g_采收_供应商[['金额']].sum())
df_供应商_采收.reset_index(inplace=True)

g_券_供应商 =df_券金额原始数据.groupby(['大类编码','供应商编码'])
df_供应商_用券金额=pd.DataFrame(g_券_供应商[['券分摊金额']].sum())
df_供应商_用券金额.reset_index(inplace=True)



In [13]:
#生成供应商的主档

df_券金额原始数据_zsj = read_file.r_csv_utf8(in_file_dir + in_file4 ,0,0)

df_券金额原始数据_zsj.rename(columns={'供应商业务码':'供应商编码'},inplace=True)




df_gys1=df_供应商原始数据.loc[:,[ '供应商编码', '供应商名称']]
df_gys2=df_采购收入原始数据.loc[:,[ '供应商编码', '供应商名称']]
df_gys3=df_券金额原始数据_zsj.loc[:,[ '供应商编码', '供应商名称']]
df_供应商主档 =df_gys1.append([df_gys2,df_gys3])

df_供应商主档.drop_duplicates(subset = ['供应商编码'],inplace=True)

In [14]:
df_供应商_1=pd.merge(df_供应商_销售,df_供应商_采收,on=['大类编码','供应商编码'],how='outer')
df_供应商_2=pd.merge(df_供应商_1,df_供应商_用券金额,on=['大类编码','供应商编码'],how='outer')

df_供应商_out_tmp=pd.merge(df_供应商_2,df_供应商主档,on=['供应商编码'],how='left')

df_供应商_out_tmp.rename(columns={'券分摊金额':'万家承担金额（去税）'},inplace=True)
df_供应商_out_tmp=df_供应商_out_tmp.fillna(0)



## 导出供应商的数据

In [15]:

df_供应商_out_tmp['券后总毛利']=df_供应商_out_tmp['月-销售毛利']-df_供应商_out_tmp['万家承担金额（去税）']+df_供应商_out_tmp['金额']

df_供应商_out_tmp['月-销售净额'] = round(df_供应商_out_tmp['月-销售净额'],2)
df_供应商_out_tmp['月-销售毛利'] = round(df_供应商_out_tmp['月-销售毛利'],2)
df_供应商_out_tmp['金额'] = round(df_供应商_out_tmp['金额'],2)
df_供应商_out_tmp['万家承担金额（去税）'] = round(df_供应商_out_tmp['万家承担金额（去税）'],2)
df_供应商_out_tmp['券后总毛利'] = round(df_供应商_out_tmp['券后总毛利'],2)

df_供应商_out_tmp.rename(columns={'金额':'采购收入分摊'},inplace=True)
df_供应商_out_tmp.rename(columns={'万家承担金额（去税）':'用券金额分摊'},inplace=True)


df_供应商_out=df_供应商_out_tmp.loc[:,['大类编码','供应商编码','供应商名称','月-销售净额','月-销售毛利','采购收入分摊','用券金额分摊', \
                                            '券后总毛利']]

print('data work finish')

data work finish


In [16]:
#导出直接给到业务部门的xlsx
writer=pd.ExcelWriter(out_file_dir+out_file)
df_供应商_out.to_excel(writer,header=True,sheet_name='供应商毛利分析',index=False)
writer.save()
print('xls格式导出 over')

xls格式导出 over


In [18]:
writer.close()